# <font color='black'> Camera Program</font>
This program aimed to control DLSR and Blackfly GigE cameras .<br/>

It starts by importing Python and custom modules

In [ ]:
import os
import sys
import sftp
import time
import getpass
import winsound
import numpy as np
import pandas as pd
from datetime import datetime

import capture

### Directories 
Define the folder paths where is located the green fabric datasets

In [ ]:
local = 'F:/pheno/'                             # Images are stored in local PC
server = '/data/phenomics-archive/whelanlab/'   # Images are stored in Remote server
EXP = 'Exp' + '_' + expNumb + '_' + expName
remote = server + EXP + '/'

### Variable declaration 
Define the Experiment & camera ID details

- Provide for the SHH credentials including username and password. It uses the Secure File Transfer Protocol (SFTP) to transfer files between local computer and the server.

In [ ]:
#%% Login LIMS-HPC
username = 'dlozano'
df = pd.read_csv('pw.csv')
password = df.columns.values[0]

- Provide the experiment details:

In [ ]:
expNumb = '12'
expName = 'RAO_dns'


- Set the time for acquiring images. It includes the start, end and the period

In [ ]:
acqStart = '05:20'      # Start time for acqusition
acqFinish = '20:40'     # End time for acqusition
acqPeriod = '15'        # Period in minutes

strtme = int(acqStart.split(':')[0]) *60 + int(acqStart.split(':')[1])
fshtme = int(acqFinish.split(':')[0]) *60 + int(acqFinish.split(':')[1])
vecTime = np.arange(strtme, fshtme, int(acqPeriod))


In [ ]:
#%% Beep sound
def beeping(t):
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = t * 1000  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)

In [ ]:
#%% Check membership
flag = False
flag = sftp.loginHPC(username, password)
if flag == False:
    sys.exit('Incorrect username or password')

In [ ]:

while True:
    now = time.localtime()
    year = now[0]
    month = now[1]
    day = now[2]
    hour = now[3]
    mins = now[4]
    second = now[5]
    
    yy = str(format(year, '04d'))
    mon = str(format(month, '02d'))
    dd =  str(format(day, '02d'))
    hh = str(format(hour, '02d'))
    mm = str(format(mins, '02d'))
    sec = str(format(second, '02d'))
    date = np.str(yy)+'-'+np.str(mon)+'-'+np.str(dd)+'-'+np.str(hh)+'-'+np.str(mm)+'-'+np.str(sec)


    minstart = vecTime[np.where(vecTime == (hour*60+mins))]
    closeInd = np.argmin(np.abs(vecTime - (hour*60+mins)))
    closeTime = vecTime[closeInd]

    if closeTime < (hour*60+mins):
        closeTime = vecTime[closeInd + 1]
        
    if (len(minstart)>0):
        capture.capture(local, date)
        closeTime = vecTime[closeInd + 2]
        
        print(date, '      ', 'Next taken in ', acqPeriod , 'mins')
        print(' ')
        print('Acquisition is running')
        time.sleep(60)

    if (closeTime - (hour*60+mins)) <= 1:
        beeping(10)
        time.sleep(1)
    
    if (closeTime - (hour*60+mins))% 5 == 0:
        time.sleep(40)
        print(date, '      ', 'Next taken in ', closeTime - (hour*60+mins), 'mins')  
    
        
    if ((hour*60+mins) > np.max(vecTime) + 10*60):
        print('No sever connection')
        sftp.upload(username, password, local, remote, expNumb)